In [1]:
import os
from woocommerce import API
from dotenv import load_dotenv

load_dotenv()

woocommerce_url = os.getenv('URL')
woocommerce_key = os.getenv('KEY')
woocommerce_secret = os.getenv('SECRET')

wc_api = API(
    url=woocommerce_url,
    consumer_key=woocommerce_key,
    consumer_secret=woocommerce_secret,
    version='wc/v3'
)


In [2]:
params = {
    'search': 'SAUCONY',
    'status': 'publish',
    'password_protected': False,
    'on_sale': 'true',
    'per_page': 95
}


In [7]:
all_saucony_products = []

page = 1
while True:
    params['page'] = page
    products = wc_api.get("products", params=params).json()
    if not products:
        break
    all_saucony_products.extend(products)
    page += 1


In [12]:
originals_sku_list = []
originals_list = []
originals_id_list = []

for product in all_saucony_products:
    attributes = product.get('attributes', [])
    for attribute in attributes:
        if attribute.get('name') == 'Brend' and 'Saucony Originals' in attribute.get('options', []):
            originals_id_list.append(product['id'])

In [11]:
for product in originals_list:
    print(product['date_created'])

2023-03-01T22:42:14
2023-03-01T22:42:12
2023-03-01T22:42:12
2023-03-01T22:42:10
2023-03-01T22:42:10
2023-03-01T22:42:07
2023-03-01T22:42:07
2023-03-01T22:42:05
2023-03-01T22:42:05
2023-03-01T22:42:03
2023-03-01T22:42:03
2023-03-01T22:41:53
2023-03-01T14:56:30
2023-03-01T14:56:27
2023-03-01T14:56:27
2023-03-01T14:56:25
2023-03-01T14:56:25
2023-03-01T14:56:25
2023-03-01T14:56:24
2023-02-28T14:56:30
2023-02-28T14:56:20
2023-02-28T14:56:20
2023-02-28T14:56:18
2023-02-28T14:56:18
2023-02-28T14:56:16
2023-02-28T14:56:16
2023-02-28T14:56:14
2023-02-28T14:56:14
2023-02-08T15:54:08
2023-02-08T15:54:08
2023-02-08T15:53:54
2023-02-08T15:53:54
2023-02-08T15:53:54
2023-02-06T00:00:00
2023-02-06T00:00:00
2023-02-06T00:00:00
2023-02-06T00:00:00
2023-02-06T00:00:00
2023-02-06T00:00:00
2023-02-06T00:00:00
2023-02-06T00:00:00
2023-02-06T00:00:00
2023-02-06T00:00:00
2023-02-06T00:00:00
2023-02-06T00:00:00
2023-02-06T00:00:00
2023-02-06T00:00:00
2023-02-06T00:00:00
2023-02-06T00:00:00
2023-02-06T00:00:00


In [ ]:
wc_api.get("products", params={"per_page": 20}).json()

In [30]:
product_sku = '006272-108'
product = wc_api.get('products', params={"sku":product_sku}).json()

In [16]:
low_originals = []

for product_id in originals_id_list:
    total_stock = 0 
    variations = wc_api.get(f"products/{product_id}/variations").json()

    for variation in variations:
        stock_quantity = variation['stock_quantity']
        total_stock += stock_quantity if stock_quantity is not None else 0
    
    if total_stock <= 3:
        low_originals.append(product_id)


In [22]:
print(originals_id_list)

[10862, 10838, 10829, 10811, 10798, 10790, 10783, 10768, 10759, 10749, 10737, 10636, 18478, 18458, 18447, 18429, 18423, 18418, 18407, 18468, 18369, 18358, 18350, 18335, 18325, 18319, 18309, 18290, 9360, 9351, 9213, 9204, 9193, 14689, 14677, 14673, 14669, 14666, 14661, 14646, 14644, 14627, 14618, 14616, 14614, 14573, 14570, 14542, 14503, 14499, 14495, 14477, 14207, 14205, 14201, 14197, 14189, 14181, 14154, 14122, 14117, 14110, 14102, 14099, 14051, 14049, 14046, 14042, 14034, 14028, 14026, 14022, 14020, 14016, 14013, 14004, 11030, 11025, 11018, 11008, 7832, 7818, 7809, 7796, 7788, 7781, 7769, 7760, 7751, 7742, 7734, 6590, 6582, 6566, 6559, 6551, 6544, 6534, 6525, 6520, 6514, 6506, 10311, 10298, 10848]


In [59]:
print(low_originals)

[14689, 14677, 14673, 14669, 14666, 14646, 14644, 14618, 14616, 14614, 14573, 14570, 14503, 14477, 14207, 14205, 14117, 14051, 14049, 14046, 14042, 14028, 14026, 14016, 14013, 6590, 6566, 6559, 6551, 6520, 10298]


In [52]:
def add_categories_to_product(product_id, new_category_ids):
    product = wc_api.get(f"products/{product_id}").json()
    existing_category_ids = [category['id'] for category in product['categories']]
    
    categories_to_update = list(set(existing_category_ids + new_category_ids))
    
    updated_categories = [{'id': cat_id} for cat_id in categories_to_update]

    updated_product = {'categories': updated_categories}
    print(wc_api.put(f"products/{product_id}", data=updated_product))

In [ ]:
category_ids_to_add = [151, 132, 152]

In [ ]:
for single_original in low_originals:
  add_categories_to_product(single_original, category_ids_to_add)

In [60]:
data = {
    "date_created": "2017-03-23T17:01:14"
}
print(wc_api.put("products/14689", data))

<Response [200]>


In [71]:
def get_product_price(product_id):
    product = wc_api.get(f"products/{product_id}").json()
    return float(product['price']) if 'price' in product else 0.0

In [72]:
sorted_product_ids = sorted(low_originals[:5], key=get_product_price, reverse=True)

In [ ]:
wc_api.get("products/14690").json()

In [84]:
product_id = 14689

product = wc_api.get(f"products/{product_id}").json()

if 'variations' in product and len(product['variations']) > 0:
    variation_id = product['variations'][0]
    variation = wc_api.get(f"products/{variation_id}").json()

    regular_price = float(
        variation['regular_price']) if 'regular_price' in variation else 0.0
    sale_price = float(variation['sale_price']
                       ) if 'sale_price' in variation else 0.0

    if regular_price != 0:
        percentage_difference = (
            (regular_price - sale_price) / regular_price) * 100
        print(
            f"The percentage difference between regular price and sale price is: {percentage_difference:.2f}%")
    else:
        print("Regular price is not available.")
else:
    print("Product has no variations.")


The percentage difference between regular price and sale price is: 50.00%


In [93]:
percentage_ranges = {
    '0-10': [],
    '11-20': [],
    '21-30': [],
    '31-40': [],
    '41-50': [],
    '51-60': [],
    '61-70': [],
    '71-80': [],
}

# Function to calculate the percentage difference


def calculate_percentage_difference(regular_price, sale_price):
    if regular_price != 0:
        return ((regular_price - sale_price) / regular_price) * 100
    return 0.0


# Categorize products into percentage difference ranges
for product_id in originals_id_list:
    product = wc_api.get(f"products/{product_id}").json()
    if 'variations' in product and len(product['variations']) > 0:
        variation_id = product['variations'][0]
        variation = wc_api.get(f"products/{variation_id}").json()

        regular_price = float(
            variation['regular_price']) if 'regular_price' in variation else 0.0
        sale_price = float(variation['sale_price']
                           ) if 'sale_price' in variation else 0.0

        percentage_difference = calculate_percentage_difference(
            regular_price, sale_price)

        for range_name, range_values in percentage_ranges.items():
            start, end = map(int, range_name.split('-'))
            if start <= percentage_difference <= end:
                range_values.append(product_id)
                break
    print(f'done with {product_id}')


done with 10862
done with 10838
done with 10829
done with 10811
done with 10798
done with 10790
done with 10783
done with 10768
done with 10759
done with 10749
done with 10737
done with 10636
done with 18478
done with 18458
done with 18447
done with 18429
done with 18423
done with 18418
done with 18407
done with 18468
done with 18369
done with 18358
done with 18350
done with 18335
done with 18325
done with 18319
done with 18309
done with 18290
done with 9360
done with 9351
done with 9213
done with 9204
done with 9193
done with 14689
done with 14677
done with 14673
done with 14669
done with 14666
done with 14661
done with 14646
done with 14644
done with 14627
done with 14618
done with 14616
done with 14614
done with 14573
done with 14570
done with 14542
done with 14503
done with 14499
done with 14495
done with 14477
done with 14207
done with 14205
done with 14201
done with 14197
done with 14189
done with 14181
done with 14154
done with 14122
done with 14117
done with 14110
done with 141

In [95]:
print(percentage_ranges)

{'0-10': [], '11-20': [10862], '21-30': [10838, 10811, 10783, 10768, 10759, 10749, 10737, 10636, 18468, 18369, 18358, 18350, 18335, 18290, 9360, 9351, 9213, 9204, 9193, 14034, 14004, 11030, 11008, 10848], '31-40': [10829, 10798, 10790, 18478, 18407, 18325, 18319, 14189, 14181, 14046, 6582, 6566, 6551, 6544, 6534, 6525, 6520, 6514, 6506, 10311, 10298], '41-50': [18418, 14677, 14661, 14627, 14197, 14122, 14117, 14051, 7809, 7788, 7751], '51-60': [14499, 14495, 14477], '61-70': [], '71-80': []}
